In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_curve, 
    auc, 
    precision_recall_curve, 
    average_precision_score
)
import shutil

# Memory and GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

class WeatherClassificationPipeline:
    def __init__(self, dataset_path, output_path, target_size=(224, 224), sample_size=30000):
        self.dataset_path = dataset_path
        self.output_path = output_path
        self.target_size = target_size
        self.sample_size = sample_size
        
        # Predefined classes to ensure consistency
        self.classes = ['clear', 'overcast', 'partly cloudy', 'rainy', 'snowy', 'unknown']

    def extract_sample_dataset(self):
        """Extract a stratified sample of images for train and validation"""
        print("\n--- STEP: Extracting Sample Dataset ---")
        
        # Create output directories
        sample_train_path = os.path.join(self.output_path, 'train')
        sample_val_path = os.path.join(self.output_path, 'val')
        os.makedirs(sample_train_path, exist_ok=True)
        os.makedirs(sample_val_path, exist_ok=True)
        
        # Calculate total samples and samples per class for train and val
        train_sample_ratio = 0.8
        val_sample_ratio = 0.2
        total_samples_per_class = self.sample_size // len(self.classes)
        train_samples_per_class = int(total_samples_per_class * train_sample_ratio)
        val_samples_per_class = int(total_samples_per_class * val_sample_ratio)

        for cls in self.classes:
            # Create class directories
            train_cls_path = os.path.join(sample_train_path, cls)
            val_cls_path = os.path.join(sample_val_path, cls)
            os.makedirs(train_cls_path, exist_ok=True)
            os.makedirs(val_cls_path, exist_ok=True)

            # Find source directory
            cls_source_path = os.path.join(self.dataset_path, 'train', cls)
            
            # Get all image files
            image_files = [f for f in os.listdir(cls_source_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            # Randomly shuffle images
            np.random.shuffle(image_files)
            
            # Select train and validation samples
            train_images = image_files[:train_samples_per_class]
            val_images = image_files[train_samples_per_class:train_samples_per_class+val_samples_per_class]
            
            # Copy train images
            for img in train_images:
                src = os.path.join(cls_source_path, img)
                dst = os.path.join(train_cls_path, img)
                shutil.copy(src, dst)
            
            # Copy validation images
            for img in val_images:
                src = os.path.join(cls_source_path, img)
                dst = os.path.join(val_cls_path, img)
                shutil.copy(src, dst)
            
            print(f"Class {cls}:")
            print(f"  Train images: {len(train_images)}")
            print(f"  Validation images: {len(val_images)}")

        return self.output_path

    def prepare_data_generators(self, dataset_path):
        """Prepare data generators for training and validation"""
        print("\n--- STEP: Preparing Data Generator ---")
        
        # Create ImageDataGenerator with preprocessing and augmentation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )

        val_datagen = ImageDataGenerator(
            rescale=1./255
        )

        # Train generator
        train_generator = train_datagen.flow_from_directory(
            os.path.join(dataset_path, 'train'),
            target_size=self.target_size,
            batch_size=32,
            class_mode='categorical',
            shuffle=True,
            classes=self.classes
        )

        # Validation generator
        val_generator = val_datagen.flow_from_directory(
            os.path.join(dataset_path, 'val'),
            target_size=self.target_size,
            batch_size=32,
            class_mode='categorical',
            shuffle=False,
            classes=self.classes
        )
        
        print(f"Classes: {self.classes}")
        print(f"Number of training samples: {train_generator.samples}")
        print(f"Number of validation samples: {val_generator.samples}")

        return train_generator, val_generator

    def create_resnet_model(self):
        """Create custom ResNet-like model with fixed downsampling"""
        def resnet_block(x, filters, kernel_size=3, downsample=False):
            # Store original input for skip connection
            shortcut = x
            
            # First convolution layer
            x = layers.Conv2D(filters, kernel_size, padding='same', 
                              strides=2 if downsample else 1)(x)
            x = layers.BatchNormalization()(x)
            x = layers.Activation('relu')(x)
            
            # Second convolution layer
            x = layers.Conv2D(filters, kernel_size, padding='same')(x)
            x = layers.BatchNormalization()(x)
            
            # Adjust shortcut if downsampling
            if downsample:
                shortcut = layers.Conv2D(filters, 1, strides=2, padding='same')(shortcut)
                shortcut = layers.BatchNormalization()(shortcut)
            
            # Merge shortcut and main path
            x = layers.Add()([x, shortcut])
            x = layers.Activation('relu')(x)
            return x

        # Input layer
        inputs = layers.Input(shape=(*self.target_size, 3))
        
        # Initial convolution and pooling
        x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

        # ResNet Blocks with proper downsampling
        x = resnet_block(x, 64)
        x = resnet_block(x, 128, downsample=True)
        x = resnet_block(x, 256, downsample=True)

        # Global average pooling and dense layers
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(len(self.classes), activation='softmax')(x)

        model = models.Model(inputs=inputs, outputs=outputs)
        return model

    def plot_training_curves(self, history):
        """Plot training and validation accuracy/loss"""
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_curves.png')
        plt.close()

    def plot_confusion_matrix(self, val_generator, model):
        """Plot confusion matrix"""
        # Get predictions
        predictions = model.predict(val_generator)
        y_pred = np.argmax(predictions, axis=1)
        y_true = val_generator.classes

        # Plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', 
                    xticklabels=self.classes, 
                    yticklabels=self.classes)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.tight_layout()
        plt.savefig('confusion_matrix.png')
        plt.close()

    def plot_roc_curve(self, val_generator, model):
        """Plot ROC curves for multi-class classification"""
        y_pred_proba = model.predict(val_generator)
        y_true = val_generator.classes
        
        plt.figure(figsize=(10, 8))
        
        # Compute ROC curve and ROC area for each class
        n_classes = len(self.classes)
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        # Binarize the output
        y_true_bin = to_categorical(y_true, num_classes=n_classes)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            plt.plot(fpr[i], tpr[i], 
                     label=f'ROC curve (class: {self.classes[i]}, area = {roc_auc[i]:.2f})')
        
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig('roc_curve.png')
        plt.close()

    def plot_precision_recall_curve(self, val_generator, model):
        """Plot Precision-Recall curves for multi-class classification"""
        y_pred_proba = model.predict(val_generator)
        y_true = val_generator.classes
        
        plt.figure(figsize=(10, 8))
        
        # Compute Precision-Recall curve
        n_classes = len(self.classes)
        y_true_bin = to_categorical(y_true, num_classes=n_classes)
        
        for i in range(n_classes):
            precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_pred_proba[:, i])
            avg_precision = average_precision_score(y_true_bin[:, i], y_pred_proba[:, i])
            
            plt.plot(recall, precision, 
                     label=f'Precision-Recall curve (class: {self.classes[i]}, AP = {avg_precision:.2f})')
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend(loc="best")
        plt.tight_layout()
        plt.savefig('precision_recall_curve.png')
        plt.close()

    def generate_classification_report(self, val_generator, model):
        """Generate and save detailed classification report"""
        y_pred_proba = model.predict(val_generator)
        y_pred = np.argmax(y_pred_proba, axis=1)
        y_true = val_generator.classes
        
        # Generate classification report
        report = classification_report(
            y_true, 
            y_pred, 
            target_names=self.classes, 
            output_dict=True
        )
        
        # Create a DataFrame for better visualization
        report_df = pd.DataFrame(report).transpose()
        
        # Plot as heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(report_df.iloc[:-2, :-1].astype(float), 
                    annot=True, 
                    cmap='YlGnBu', 
                    fmt='.2f')
        plt.title('Classification Report Metrics')
        plt.tight_layout()
        plt.savefig('classification_report_heatmap.png')
        plt.close()
        
        # Save textual report
        with open('classification_report.txt', 'w') as f:
            f.write(classification_report(
                y_true, 
                y_pred, 
                target_names=self.classes
            ))
        
        return report  

    def visualize_sample_images(self, dataset_path):
        """Visualize sample images from each class"""
        print("\n--- STEP: Visualizing Sample Images ---")
        
        # Create figure for sample images
        plt.figure(figsize=(15, 10))
        
        # Get train directory
        train_dir = os.path.join(dataset_path, 'train')
        
        # Iterate through classes
        for i, cls in enumerate(self.classes, 1):
            # Get path to class directory
            cls_path = os.path.join(train_dir, cls)
            
            # Get list of image files
            image_files = [f for f in os.listdir(cls_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            # Select first image
            if image_files:
                img_path = os.path.join(cls_path, image_files[0])
                
                # Load and display image
                plt.subplot(2, 3, i)
                img = plt.imread(img_path)
                plt.imshow(img)
                plt.title(cls)
                plt.axis('off')
        
        plt.tight_layout()
        plt.savefig('sample_images.png')
        plt.close()

    def predict_single_image(self, model, image_path):
        """Predict weather class for a single image"""
        print("\n--- STEP: Predicting Single Image ---")
        
        # Load and preprocess the image
        img = load_img(image_path, target_size=self.target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize
        
        # Make prediction
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class = self.classes[predicted_class_index]
        confidence = predictions[0][predicted_class_index]
        
        # Visualize prediction
        plt.figure(figsize=(10, 5))
        
        # Original image
        plt.subplot(1, 2, 1)
        plt.imshow(plt.imread(image_path))
        plt.title('Original Image')
        plt.axis('off')
        
        # Prediction bar plot
        plt.subplot(1, 2, 2)
        plt.bar(self.classes, predictions[0])
        plt.title('Class Probabilities')
        plt.xlabel('Weather Classes')
        plt.ylabel('Probability')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.savefig('single_image_prediction.png')
        plt.close()
        
        # Print prediction details
        print(f"Predicted Class: {predicted_class}")
        print(f"Confidence: {confidence:.2%}")
        
        # Print full probabilities
        for cls, prob in zip(self.classes, predictions[0]):
            print(f"{cls}: {prob:.2%}")
        
        return predicted_class, confidence
           
    def run_pipeline(self):
        # Extract sample dataset
        sample_train_path = self.extract_sample_dataset()

        # Visualize sample images
        self.visualize_sample_images(sample_train_path)

        # Prepare data generators
        train_generator, val_generator = self.prepare_data_generators(sample_train_path)

        # Create model
        model = self.create_resnet_model()
        
        # Compile model
        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Callbacks
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2, 
            patience=5, 
            min_lr=0.00001
        )
        
        early_stopping = EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )

        # Training
        history = model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=50,
            callbacks=[reduce_lr, early_stopping]
        )

        # Visualizations
        self.plot_training_curves(history)
        self.plot_confusion_matrix(val_generator, model)
        self.plot_roc_curve(val_generator, model)
        self.plot_precision_recall_curve(val_generator, model)
        report = self.generate_classification_report(val_generator, model)

        # Predict single image
        test_image_path = '/kaggle/input/bdd100k-weather-classification/test/cabc30fc-e7726578.jpg'
        self.predict_single_image(model, test_image_path)
        
        # Print out some key metrics from the report
        print("\nClassification Report Summary:")
        for cls in self.classes:
            print(f"{cls}:")
            print(f"  Precision: {report[cls]['precision']:.4f}")
            print(f"  Recall: {report[cls]['recall']:.4f}")
            print(f"  F1-Score: {report[cls]['f1-score']:.4f}")

        # Evaluate model
        val_loss, val_accuracy = model.evaluate(val_generator)
        print(f"\nValidation Accuracy: {val_accuracy}")

        # Save model
        model.save('weather_classification_model.h5')
        
        return model

# Main execution
if __name__ == "__main__":
    # Create output directory for sample dataset
    output_path = '/kaggle/working/bdd100k_sample'
    os.makedirs(output_path, exist_ok=True)

    # Initialize and run pipeline
    pipeline = WeatherClassificationPipeline(
        dataset_path='/kaggle/input/bdd100k-weather-classification',
        output_path=output_path
    )
    pipeline.run_pipeline()


--- STEP: Extracting Sample Dataset ---
Class clear:
  Train images: 4000
  Validation images: 1000
Class overcast:
  Train images: 4000
  Validation images: 1000
Class partly cloudy:
  Train images: 4000
  Validation images: 881
Class rainy:
  Train images: 4000
  Validation images: 1000
Class snowy:
  Train images: 4000
  Validation images: 1000
Class unknown:
  Train images: 4000
  Validation images: 1000

--- STEP: Visualizing Sample Images ---

--- STEP: Preparing Data Generator ---
Found 24000 images belonging to 6 classes.
Found 5881 images belonging to 6 classes.
Classes: ['clear', 'overcast', 'partly cloudy', 'rainy', 'snowy', 'unknown']
Number of training samples: 24000
Number of validation samples: 5881
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734460239.338342     105 service.cc:145] XLA service 0x7e03d4108620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734460239.338412     105 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734460239.338417     105 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  2/750 ━━━━━━━━━━━━━━━━━━━━ 56s 76ms/step - accuracy: 0.2031 - loss: 1.9970   

I0000 00:00:1734460249.452922     105 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


750/750 ━━━━━━━━━━━━━━━━━━━━ 352s 445ms/step - accuracy: 0.3699 - loss: 1.5312 - val_accuracy: 0.2836 - val_loss: 2.0467 - learning_rate: 0.0010
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 350s 462ms/step - accuracy: 0.4461 - loss: 1.3537 - val_accuracy: 0.2998 - val_loss: 1.8152 - learning_rate: 0.0010
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 336s 444ms/step - accuracy: 0.4737 - loss: 1.3055 - val_accuracy: 0.4574 - val_loss: 1.2980 - learning_rate: 0.0010
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 331s 437ms/step - accuracy: 0.4926 - loss: 1.2671 - val_accuracy: 0.3328 - val_loss: 1.5561 - learning_rate: 0.0010
Epoch 5/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 356s 471ms/step - accuracy: 0.5043 - loss: 1.2268 - val_accuracy: 0.3940 - val_loss: 1.5229 - learning_rate: 0.0010
Epoch 6/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 354s 468ms/step - accuracy: 0.5406 - loss: 1.1483 - val_accuracy: 0.4594 - val_loss: 1.3213 - learning_rate: 0.0010
Epoch 7/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 356s 471ms/step - accuracy: 0.5667 